In [ ]:
# TODO:: 

# 1. build date function and convert utc to mm dd - done!!!
# 1.1 - instead of using date select miliseconds as test data from 11.2018 / check if at least 20% of data

# 2. remove duplicate items per single person / remove by bidid  - done my michael - done!!!

# 3. remove trash columns from android_bids
# 3.1 remove marketplace col - done !!!
# 3.2 device_maker and device_model NaN's changed to unknown and claasified as new device_maker or device_model - done!!!

# 4. remove trash columns from play_apps
# 4.1 Create dataframe from files - done!!!
# 4.2 Do we need to use play_apps -> Which cols do we want to use??
#  [video, category, score, histogram, reviews, editors_choice, (price, free), iap, size, installs, content_rating, iap_range, interactive_elements?, bids]

# 5. trun into comma seperated label for play_apps -> category

# 6. Merge data from two tables android_bids_us and play_apps
# 6.1 USE pandas merge datafreames function to merge both files

In [ ]:
#imports
import pandas as pd
from datetime import datetime
from zipfile import ZipFile
import pickle


In [ ]:
#Read orig android data
origdata = pd.read_csv('data/android_bids_us.csv')
origdata.head()


In [ ]:
#Read orig play_apps data
app_details_orig = ZipFile("data/play_apps.zip")
app_file = 'play_apps/a008.com.fc2.blog.androidkaihatu.datecamera2'
app = pickle.loads(app_details_orig.read(app_file))
app

In [ ]:
dataTypes = origdata.dtypes
print(f'Total columns: {dataTypes.size} ||| Total rows: {origdata.count()}')
print(f'Data type of each column of Dataframe : ')
print(dataTypes)
# We may have NAN's in user_isp, device_maker, device_model

print('-----------------------------------')
print(origdata.marketplace.unique())
# all items in marketplace col equal 'chartboost'. we can remove col
print('-----------------------------------')

print('-----------------------------------')
print('app_id::')
print(origdata.app_id.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_ip::')
print(origdata.user_isp.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_state::')
print(origdata.user_state.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_maker::')
print(origdata.device_maker.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_model::')
print(origdata.device_model.value_counts())
print('-----------------------------------')



In [ ]:
#Rearrange dataframe
dataframe = origdata

#-----------------------------

#Add date from utc and remove utc
dataframe['date'] = dataframe['utc_time'].apply(lambda x: datetime.fromtimestamp(x/1000))
dataframe = dataframe.drop(['utc_time'], axis=1)

#-----------------------------

#Remove marketplace col
dataframe = dataframe.drop(['marketplace'], axis=1)

#-----------------------------

#Remove duplicated bidid. Keep only single instance keep the one with click == 1
dataframe = dataframe[~((dataframe['bidid'].duplicated(keep=False))&(dataframe['click']==0))]
#and then remove bids column
dataframe = dataframe.drop(['bidid'], axis=1)

#-----------------------------

#Change device_maker NaN's to unknown manufacturerer
dataframe['device_maker'] = dataframe['device_maker'].apply(lambda x: 'unknown' if x!=x else x)

#-----------------------------

#Change device_model NaN's to unknown valid model
dataframe['device_model'] = dataframe['device_model'].apply(lambda x: 'unknown' if x!=x else x)

#-----------------------------

dataframe

In [ ]:
# Get play_apps data

def zipToDataFrame(zipped, to_remove):
    rows = []
    
    for f in zipped.filelist:
        app_file = f.filename
        try:
            app = pickle.loads(zipped.read(app_file))
        except EOFError as e:
            continue
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            return
            
        # Remove columns
        for col in to_remove:
            del app[col]
        rows.append(app)
    
    return pd.DataFrame(rows)


app_details = ZipFile("data/play_apps.zip")
cols_to_remove = ['title','screenshots', 'icon', 
                  'histogram', 'description', 'description_html', 
                  'recent_changes', 'developer', 'developer_id', 'developer_email', 'developer_url',
                  'developer_address', 'url', 'current_version', 'updated']


orig_play_apps = zipToDataFrame(app_details, cols_to_remove)
orig_play_apps



In [ ]:
# explore play_apps data characteristics

apps_dataTypes = orig_play_apps.dtypes

print(f'Total columns: {apps_dataTypes.size} ||| Total rows: {orig_play_apps.count()}')
print(f'Data type of each column of Dataframe : ')
print(apps_dataTypes)
# We may have NAN's in iap_range, interactive_elements, price

print('-----------------------------------')
print(orig_play_apps.price.unique())
print(orig_play_apps.free.unique())

# items in price and free should be in a single col

print('-----------------------------------')
